In [1]:
# Hash function: SHA256
from hashlib import sha256

def get_hash(message_string):
    return sha256(message_string.encode('utf-8')).hexdigest()

print("Hash of 'apple':", get_hash("apple"))
print("Hash of 'apples':", get_hash("apples"))
print("Hash of 'zpple':", get_hash("zpple"))

# Output:
# Hash of 'apple': 3a7bd3e2360a3d29eea436fcfb7e44c735d117c42d1c1835420b6b9942dd4f1b
# Hash of 'apples': f5903f51e341a783e69ffc2d9b335048716f5f040a782a2764cd4e728b0f74d9
# Hash of 'zpple': 2d198bd02ab5e8a7260f50db5dea5672c03151c3febcf3d2ec129263d4f299bd


Hash of 'apple': 3a7bd3e2360a3d29eea436fcfb7e44c735d117c42d1c1835420b6b9942dd4f1b
Hash of 'apples': f5903f51e341a783e69ffc2d9b335048716f5f040a782a2764cd4e728b0f74d9
Hash of 'zpple': 2d198bd02ab5e8a7260f50db5dea5672c03151c3febcf3d2ec129263d4f299bd


In [2]:
# Hash to curve
# Try-and-Increment method

from hashlib import sha256
from py_ecc.secp256k1 import secp256k1

def hash_to_curve_try_and_increment(message):
    # Counter
    counter = 0
    while True:
        # Compute the hash of the message and the counter
        data = message + counter.to_bytes(32, 'big')
        hash_digest = sha256(data).digest()

        # Calculate the x-coordinate from the hash
        x = int.from_bytes(hash_digest, 'big') % secp256k1.P

        # Try to calculate y^2 using the elliptic curve equation y^2 = x^3 + ax + b
        y_squared = (pow(x, 3, secp256k1.P) + secp256k1.A * x + secp256k1.B) % secp256k1.P
        
        # Calculate the square root of y^2 modulo p, if it exists (for secp256k1, p % 4 = 3, this method can be used)
        y = pow(y_squared, (secp256k1.P + 1) // 4, secp256k1.P)

        # If y^2 = y_squared, then y^2 is a quadratic residue, (x, y) is a point on the elliptic curve, return it
        # Otherwise, increment the counter c by 1 and try again
        if pow(y, 2, secp256k1.P) == y_squared:

            # y^2 has two square roots y and -y, we use the last bit of the hash to determine the sign of y
            sign_bit = hash_digest[-1] & 1
            if sign_bit == 1:
                y = secp256k1.P - y  # Use -y

            return (x, y)

        counter += 1

# Example
message = 'Example message'.encode('utf-8')
point_on_curve = hash_to_curve_try_and_increment(message)
print(f"Point on secp256k1 curve: {point_on_curve}")

# Output:
# Point on secp256k1 curve: (75672206050705717597513752332592681441562708170391675094677140490692301502235, 2079954378639550643990441934946533201466680551826647417569849183466717633402)


Point on secp256k1 curve: (75672206050705717597513752332592681441562708170391675094677140490692301502235, 2079954378639550643990441934946533201466680551826647417569849183466717633402)


In [3]:
from py_ecc.secp256k1 import secp256k1
import os

def pedersen_hash(G, P, m, r):
    """
    Calculate the Pedersen hash using two points G and P on secp256k1.
    :param G: The generator point on secp256k1.
    :param P: The point obtained through hash-to-curve.
    :param m: The message.
    :param r: The random number.
    :return: The Pedersen hash as a point on the secp256k1 curve.
    """
    hash_point = secp256k1.add(secp256k1.multiply(G, m), secp256k1.multiply(P, r))
    return hash_point

def message_to_int(message):
    """Convert the message to an integer."""
    message_hash = sha256(message.encode('utf-8')).digest()
    return int.from_bytes(message_hash, 'big')

# Example
G = secp256k1.G
P = (75672206050705717597513752332592681441562708170391675094677140490692301502235, 2079954378639550643990441934946533201466680551826647417569849183466717633402)
m = message_to_int('Example message')
r = int.from_bytes(os.urandom(32), 'big') % secp256k1.N  # Random blinding factor
print(f"Random number r: {r}")
# Output:
# Random number r: 73024396533133913522992000700997978493545300935845156062873113842229351634806

hash_point = pedersen_hash(G, P, m, r)
print(f"Pedersen hash: {hash_point}")
# Output:
# Pedersen hash: (61158186220483422500680232915115660637414963516696475985433668859480078549697, 19925636506537717141567615179623096875787300022679479682581994441236159884610)


Random number r: 91532787645296465305977546857153060360042718695392396756112438925549267677625
Pedersen hash: (33207901400245673317772404642820378824040002129405689006696541902994587356526, 92276320545487270630584006967742200532131477233818169189560270074605054126669)
